In [1]:
import pandas as pd
import numpy as np

from quantfreedom.enums.enums import OrderType
from quantfreedom.plotting.plot_helper_functions import (
    get_candle_trace_data,
    append_to_trace_data_list,
)

import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, dash_table
from jupyter_dash import JupyterDash
from plotly.subplots import make_subplots
from dash_bootstrap_templates import load_figure_template

from quantfreedom.utils import clear_cache
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.evaluators.evaluators import eval_is_below, eval_is_above, combine_evals
prices = pd.read_csv(
    'E:/Coding/backtesters/QuantFreedom/tests/data/30min.csv', index_col='time')


In [2]:
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30, 50],
)
stoch_ind = from_talib(
    func_name='stoch',
    df_prices=prices,
    cart_product=False,
    combos=True,
    fastk_period=[5, 10, 15],
)
stoch_ind_slowk= stoch_ind['slowk']

In [3]:
ema_ind_on_rsi = from_talib(
    func_name='ema',
    user_ind_df=rsi_ind,
    cart_product=False,
    combos=False,
    timeperiod=[5, 20, 30]
)
ema_ind_on_stoch = from_talib(
    func_name='ema',
    user_ind_df=stoch_ind_slowk,
    cart_product=False,
    combos=False,
    timeperiod=[20, 40, 60, 100]
)

In [4]:
ema_below_rsi = eval_is_below(
    want_to_evaluate=ema_ind_on_rsi,
    indicator_data=rsi_ind,
)
ema_above_stoch_slowk = eval_is_above(
    want_to_evaluate=ema_ind_on_stoch,
    indicator_data=stoch_ind_slowk,
)

In [5]:
combos_finished = combine_evals(
    bigger_df=ema_above_stoch_slowk,
    smaller_df=ema_below_rsi,
)

In [6]:
combos_finished

STOCH_fastk_period       5                                                    \
STOCH_slowk_period        3                                                    
STOCH_slowk_matype        0                                                    
STOCH_slowd_period        3                                                    
STOCH_slowd_matype        0                                                    
STOCH_EMA_timeperiod    20                                                     
EMA_is_above          STOCH                                                    
RSI_timeperiod           15                   30                   50          
RSI_EMA_timeperiod       5      20     30     5      20     30     5      20   
EMA_is_below            RSI    RSI    RSI    RSI    RSI    RSI    RSI    RSI   
time                                                                           
2020-10-01 00:00:00   False  False  False  False  False  False  False  False   
2020-10-01 00:30:00   False  False  False  False  False  False  False  False   
2020-10-01 01:00:00   False  False  False  False  False  False  False  False   
2020-10-01 01:30:00   False  False  False  False  False  False  False  False   
2020-10-01 02:00:00   False  False  False  False  False  False  False  False   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
2021-02-22 21:30:00   False  False  False  False  False  False  False  False   
2021-02-22 22:00:00   False  False  False  False  False  False  False  False   
2021-02-22 22:30:00   False   True   True  False   True   True  False   True   
2021-02-22 23:00:00   False   True   True  False   True   True  False   True   
2021-02-22 23:30:00    True   True   True   True   True   True   True   True   

STOCH_fastk_period                  ...     15                              \
STOCH_slowk_period                  ...      3                               
STOCH_slowk_matype                  ...      0                               
STOCH_slowd_period                  ...      3                               
STOCH_slowd_matype                  ...      0                               
STOCH_EMA_timeperiod           40   ...    60     100                        
EMA_is_above                 STOCH  ...  STOCH  STOCH                        
RSI_timeperiod                  15  ...     50     15                   30   
RSI_EMA_timeperiod       30     5   ...     30     5      20     30     5    
EMA_is_below            RSI    RSI  ...    RSI    RSI    RSI    RSI    RSI   
time                                ...                                      
2020-10-01 00:00:00   False  False  ...  False  False  False  False  False   
2020-10-01 00:30:00   False  False  ...  False  False  False  False  False   
2020-10-01 01:00:00   False  False  ...  False  False  False  False  False   
2020-10-01 01:30:00   False  False  ...  False  False  False  False  False   
2020-10-01 02:00:00   False  False  ...  False  False  False  False  False   
...                     ...    ...  ...    ...    ...    ...    ...    ...   
2021-02-22 21:30:00   False  False  ...  False  False  False  False  False   
2021-02-22 22:00:00   False  False  ...  False  False  False  False  False   
2021-02-22 22:30:00    True  False  ...  False  False  False  False  False   
2021-02-22 23:00:00    True  False  ...  False  False  False  False  False   
2021-02-22 23:30:00    True   True  ...  False  False  False  False  False   

STOCH_fastk_period                                       
STOCH_slowk_period                                       
STOCH_slowk_matype                                       
STOCH_slowd_period                                       
STOCH_slowd_matype                                       
STOCH_EMA_timeperiod                                     
EMA_is_above                                             
RSI_timeperiod                         50                
RSI_EMA_timeperiod       20     30     5      20     30  


In [37]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=prices.index,
            y=rsi_ind[15].values.flatten(),
            mode="lines",
            name="RSI",
        ),
        go.Scatter(
            x=prices.index,
            y=ema_ind_on_rsi[15][30].values,
            mode="lines",
            name="EMA on RSI",
        ),
        go.Scatter(
            x=prices.index,
            y=np.where(
                ema_below_rsi[(15, 30, "RSI")].values,
                ema_ind_on_rsi[(15, 30)].values,
                np.nan,
            ),
            mode="markers",
            name="Signals",
        ),
    ]
)
fig.show()